<a id="1"></a> 
# OpenStreetMap Data Case Study
**Name: Jan FOERSTER**
***

In [1]:
%matplotlib notebook

import pprint

from jfo.schema import schema
from jfo.mySQL3dbConn import mySQLLITE3
from jfo.myXML import Investigation
from jfo.myExtension import myDict2CSVTransformer

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#sample file size 80MB
#xmlfile = 'OpenStreetMap-Hamburg-Sample.osm'

#original file size378MB
xmlfile = 'OpenStreetMap-Hamburg-31.osm'

myOSMfile = Investigation(xmlfile)

<a id="11"></a>
## Map Area
***
This map is of about my hometown and it's direct regional environment, so I’m more interested to see what database querying reveals, and I’d like an opportunity to contribute to its improvement on OpenStreetMap.org by an customized extract from MapZen

- OpenStreet Map: Freie und Hansestadt Hamburg, Germany


- __[MapZen Extract](https://mapzen.com/data/metro-extracts/your-extracts/d4104a4aab6c)__

In [2]:
%%HTML
<div class="alert alert-block alert-info">
    <iframe width="100%" height="100%" frameborder="0" scrolling="no" marginheight="0" marginwidth="0" src="http://www.openstreetmap.org/export/embed.html?bbox=9.534759521484375%2C53.38291881008687%2C10.441131591796877%2C53.7438381234801&amp;layer=mapnik&amp;marker=53.56371244239144%2C9.987602233886719" style="border: 1px solid black"></iframe>
    <br/>
    <small>
        <a href="http://www.openstreetmap.org/?mlat=53.5637&amp;mlon=9.9876#map=11/53.5637/9.9876">Show Detailed Map</a>
    </small>
</div>

<a id="11"></a> 
## Data Infrastructure
***
With the method *'count_tags()'* of my class *'Investigation'*, I save the number of unique tags of chosen XML-file within a private dictionary. By applying the method *'get_Tags()'*, I receive the full information.

### Unique Tags

In [3]:
myOSMfile.count_Tags()
pprint.pprint(myOSMfile.get_Tags())

{'bounds': 1,
 'member': 84855,
 'nd': 2118748,
 'node': 1609887,
 'osm': 1,
 'relation': 6524,
 'tag': 1574672,
 'way': 300719}


In order to understand, which attributes are used by which XML-tag the method *'view_Tags_Attributes'* saves this in a further private dictionary. 

In [4]:
myOSMfile.view_Tags_Attributes()
pprint.pprint(myOSMfile.get_Tags_Attributes())

defaultdict(<type 'set'>, {'node': set(['changeset', 'uid', 'timestamp', 'lon', 'version', 'user', 'lat', 'id']), 'nd': set(['ref']), 'bounds': set(['minlat', 'maxlon', 'minlon', 'maxlat']), 'member': set(['role', 'ref', 'type']), 'tag': set(['k', 'v']), 'relation': set(['changeset', 'uid', 'timestamp', 'version', 'user', 'id']), 'way': set(['changeset', 'uid', 'timestamp', 'version', 'user', 'id']), 'osm': set(['timestamp', 'version', 'generator'])})


### Patterns in the Tags

The 'k' attribute of tag 'tag' contains different patterns. In order to identify potential bias from used characters, all content has been classidied as following by the method *'audit_Data_Chars'*:

* lower : regex pattern is valid, if tags contain only lowercase letters
* lower_colon: regex pattern is valid, if tags contain only colons
* problemchars: regex pattern is valid, if tags contain characters, which could cause issue, e.g. '=\+/&<>;\'"\?%#$@\,\. \t\r\n'
* other: regex pattern is valid, if tags contain only nothing from above

In [5]:
myOSMfile.audit_Data_Chars()
pprint.pprint(myOSMfile.get_Problematic_Chars()) 

{'lower': 789084, 'lower_colon': 736073, 'other': 49515, 'problemchars': 0}


### Users, who have contributed to OSM
The 'user' attribute of tag 'node' reveals information, which user have contributed most / least to OSM for nodes. The method *'audit_Unique_Values('node', 'user')'*. 

In [6]:
myOSMfile.audit_Unique_Values('node', 'user')
pprint.pprint(myOSMfile.sort_Dictionary(myOSMfile.get_Unique_Values())[:10])

[('fahrrad', 121258),
 ('svbr', 116237),
 ('Abendstund', 93915),
 ('sundew', 86310),
 ('Divjo', 74068),
 ('findichgut', 64352),
 (u'gl\xfchw\xfcrmchen', 60367),
 ('vademecum', 51521),
 ('Joke123', 45668),
 ('simlox', 44720)]


The 'user' attribute of tag 'way' reveals information, which user have contributed most / least to OSM for ways. The method *'audit_Unique_Values('node', 'user')'*. 

In [7]:
myOSMfile.audit_Unique_Values('way', 'user')
pprint.pprint(myOSMfile.sort_Dictionary(myOSMfile.get_Unique_Values())[:10])

[('fahrrad', 144620),
 ('svbr', 137218),
 ('Abendstund', 109168),
 ('sundew', 95625),
 ('Divjo', 90719),
 ('findichgut', 75579),
 (u'gl\xfchw\xfcrmchen', 66099),
 ('vademecum', 60115),
 ('Joke123', 53585),
 ('simlox', 50068)]


The Top10 of contributers are:
    * 'fahrrad'
    * 'svbr'
    * 'Abendstund'
    * 'sundew'
    * 'Divjo'
    * 'findichgut'
    * 'glühwürmchen'
    * 'vademecum'
    * 'Joke123'
    * 'simlox'

<a id="12"></a> 
## Problems Encountered in the Map
***
The sample file has been  loaded, parsed, analyzed, cleaned - if necessary - and saved in SQLite3 database.

<a id="121"></a>
### Phone Numbers
***
The major problem has been to have phone numbers written in a standardized format, e.g. "+49 (0)40 XXXXXXX", as some persons have indicated international prefix and some persons not. In case international prefix has been added, there hasn't been a constant formatting way for further notion.

In [8]:
myOSMfile.audit_Phone_Numbers()
pprint.pprint(myOSMfile.get_Aligned_Phone_Numbers())   

defaultdict(<type 'set'>, {'+494028787174': '+49 (0)40 28787174', '+494028806718': '+49 (0)40 28806718', '+494059355340': '+49 (0)40 59355340', '04054880575': '+49 (0)40 54880575', '+494065067790': '+49 (0)40 65067790', '04036005520': '+49 (0)40 36005520', '+494079027754': '+49 (0)40 79027754', '+49404142760': '+49 (0)40 4142760', '+4940545077': '+49 (0)40 545077', '+494078808114': '+49 (0)40 78808114', '+49406415933': '+49 (0)40 6415933', '+4940428832225': '+49 (0)40 428832225', '+494032027757': '+49 (0)40 32027757', '+494088175508': '+49 (0)40 88175508', '+49407941630': '+49 (0)40 7941630', '+4940366560': '+49 (0)40 366560', '+4940776374': '+49 (0)40 776374', '+494058915971': '+49 (0)40 58915971', '+49402514524': '+49 (0)40 2514524', '+49408317422': '+49 (0)40 8317422', '+494039805790': '+49 (0)40 39805790', '0406523906': '+49 (0)40 6523906', '+4940853510': '+49 (0)40 853510', '+494032031387': '+49 (0)40 32031387', '+494076795391': '+49 (0)40 76795391', '+49403344113340': '+49 (0)40 

<a id="122"></a>
### Postal Codes
***
All postal codes have been indicated correctly. No cleaning has been required.

In [9]:
myOSMfile.audit_Zip_Codes()
pprint.pprint(myOSMfile.get_ZIP_Codes())

defaultdict(<type 'set'>, {'21647': '21647', '22175': '22175', '22177': '22177', '25462': '25462', '22179': '22179', '25469': '25469', '22589': '22589', '22587': '22587', '22761': '22761', '22763': '22763', '22765': '22765', '20457': '20457', '22769': '22769', '21079': '21079', '21075': '21075', '21077': '21077', '21073': '21073', '21107': '21107', '21109': '21109', '20357': '20357', '21218': '21218', '20355': '20355', '20354': '20354', '22391': '22391', '20359': '20359', '22081': '22081', '22083': '22083', '22085': '22085', '22087': '22087', '22089': '22089', '20537': '20537', '20535': '20535', '22459': '22459', '22559': '22559', '20539': '20539', '21614': '21614', '22869': '22869', '22549': '22549', '21129': '21129', '22547': '22547', '25421': '25421', '22609': '22609', '22303': '22303', '22301': '22301', '22307': '22307', '22305': '22305', '22309': '22309', '21220': '21220', '21224': '21224', '21037': '21037', '20095': '20095', '22047': '22047', '22041': '22041', '22043': '22043', '

 <a id="123"></a>
### Street Names
***
All street names have been indicated correctly with only three exceptions, which have been corrected, whereas numbers have been inidcated in combination with street names. These three house numbers have been deleted.

In [10]:
myOSMfile.audit_Street_Names()
#pprint.pprint(myOSMfile.sort_Dictionary(myOSMfile.get_Street_Names()))
#pprint.pprint(myOSMfile.get_Street_Names())
#pprint.pprint('Unknown Streets %s' % + myOSMfile.get_Street_Names('unknown'))
#pprint.pprint('Unknown Streets %s' % + myOSMfile.get_Street_Names('mysterious'))
sorted(myOSMfile.get_Aligned_Street_Names().keys())

[u'ABC-Stra\xdfe',
 'Aastwiete',
 u'Abbestra\xdfe',
 'Abendrothsweg',
 u'Abteistra\xdfe',
 u'Abtstra\xdfe',
 u'Achter L\xfcttmoor',
 'Achter de Hoef',
 u'Achter de H\xf6f',
 'Achter de Weiden',
 'Achterkamp',
 'Achtern Barls',
 'Achtern Born',
 'Achtern Brack',
 'Achtern Kronskamp',
 'Achtern Moor',
 'Achtern Sand',
 'Achtern Styg',
 'Achterndiek',
 u'Ackermannstra\xdfe',
 'Ackerstieg',
 u'Adalbert-Stifter-Stra\xdfe',
 u'Adalbertstra\xdfe',
 'Adebarweg',
 'Adenauerallee',
 u'Adickesstra\xdfe',
 'Adlerhorst',
 u'Adlerstra\xdfe',
 u'Admiralit\xe4tstra\xdfe',
 u'Adolf-Wagner-Stra\xdfe',
 'Adolf-von-Elm-Hof',
 u'Adolph-Sch\xf6nfelder-Stra\xdfe',
 u'Adolphsbr\xfccke',
 'Adolphsplatz',
 u'Afrikastra\xdfe',
 'Agathe-Lasch-Weg',
 u'Agathenstra\xdfe',
 u'Agnesstra\xdfe',
 'Ahornallee',
 'Ahornkamp',
 u'Ahornstra\xdfe',
 'Ahornweg',
 'Akazienallee',
 'Akazienweg',
 'Akeleiweg',
 u'Alardusstra\xdfe',
 'Alberichstieg',
 u'Albers-Sch\xf6nberg-Stieg',
 u'Albers-Sch\xf6nberg-Weg',
 'Albershof',
 'Alb

<a id="13"></a>
## Data Overview and Additional Ideas
***

### Create SQL3Lite database to save XML content

In [13]:
#dbUdacity = mySQLLITE3('C:\\Users\\th65jt\\sqlite3\\', 'osmHamburg.db')
dbUdacity = mySQLLITE3('C:\\Users\\JanUser\\sqlite\\sqlite_windows\\', 'osmHamburg.db')

#dbUdacity.execute_SQLStatementsFile('C:\\Users\\JanUser\\Documents\\Udacity\\Sybullus\\L3 MongoDB\\Project\\OSM_Drop_Tables.sql')
#dbUdacity.commit_SQL()

#dbUdacity.execute_SQLStatementsFile('OSM_Create_Tables.sql')
#dbUdacity.import_preformatted_CSV('.\\nodes.csv', 'nodes')
#dbUdacity.import_preformatted_CSV('.\\nodes_tags.csv', 'nodes_tags')
#dbUdacity.import_preformatted_CSV('.\\ways.csv', 'ways')
#dbUdacity.import_preformatted_CSV('.\\ways_tags.csv', 'ways_tags')
#dbUdacity.import_preformatted_CSV('.\\ways_nodes.csv', 'ways_nodes')

In [14]:
dbUdacity.query_AllResults('''SELECT nodes_tags.key                                
                                , count(nodes_tags.key) as sum
                           FROM nodes_tags
                           GROUP BY nodes_tags.key
                           ORDER BY sum DESC;''')

[(u'street', 34999),
 (u'housenumber', 34616),
 (u'postcode', 28577),
 (u'city', 26874),
 (u'natural', 24232),
 (u'name', 22760),
 (u'country', 22093),
 (u'entrance', 15777),
 (u'highway', 15071),
 (u'amenity', 15049),
 (u'type', 11967),
 (u'ref', 8454),
 (u'wheelchair', 8033),
 (u'railway', 7596),
 (u'network', 6510),
 (u'shop', 5943),
 (u'emergency', 5523),
 (u'diameter', 4890),
 (u'operator', 4695),
 (u'barrier', 4190),
 (u'historic', 4111),
 (u'website', 4076),
 (u'source', 4034),
 (u'position', 3929),
 (u'opening_hours', 3901),
 (u'addr', 3755),
 (u'crossing', 3416),
 (u'phone', 3223),
 (u'bus', 3207),
 (u'signal:direction', 3177),
 (u'signal:position', 3163),
 (u'level', 2720),
 (u'public_transport', 2270),
 (u'leaf_type', 2267),
 (u'date_of_birth', 2039),
 (u'addr:city', 1904),
 (u'addr:street', 1895),
 (u'created_by', 1757),
 (u'note', 1717),
 (u'bicycle', 1583),
 (u'text', 1581),
 (u'shelter', 1550),
 (u'cuisine', 1547),
 (u'description', 1515),
 (u'check_date', 1492),
 (u'swi

### Looking for 'restaurants' in the database

In [15]:
dbUdacity.query_AllResults('''SELECT nodes_tags.value
                                    , count (nodes_tags.key) as Sum    
                                FROM nodes_tags                                    
                                WHERE nodes_tags.key = "restaurant"                                    
                                GROUP BY nodes_tags.value
                                ORDER BY Sum DESC
                            ;''')

[(u'cafe', 1), (u'yes', 1)]

In [16]:
dbUdacity.query_AllResults('''SELECT nodes_tags.value
                                    , count (nodes_tags.key) as Sum    
                                FROM nodes_tags                                    
                                WHERE nodes_tags.key = "amenity"                                    
                                GROUP BY nodes_tags.value
                                ORDER BY Sum DESC
                            ;''')

[(u'bench', 2531),
 (u'restaurant', 1527),
 (u'post_box', 1015),
 (u'vending_machine', 838),
 (u'recycling', 834),
 (u'parking', 763),
 (u'cafe', 669),
 (u'fast_food', 638),
 (u'waste_basket', 463),
 (u'bicycle_parking', 462),
 (u'telephone', 410),
 (u'pub', 383),
 (u'doctors', 361),
 (u'pharmacy', 328),
 (u'bank', 289),
 (u'kindergarten', 244),
 (u'clock', 198),
 (u'toilets', 198),
 (u'bar', 192),
 (u'bicycle_rental', 190),
 (u'parking_entrance', 184),
 (u'atm', 152),
 (u'post_office', 149),
 (u'library', 146),
 (u'taxi', 119),
 (u'fuel', 114),
 (u'dentist', 107),
 (u'charging_station', 103),
 (u'place_of_worship', 76),
 (u'social_facility', 76),
 (u'shelter', 71),
 (u'driving_school', 64),
 (u'school', 62),
 (u'fountain', 60),
 (u'community_centre', 53),
 (u'car_rental', 52),
 (u'theatre', 52),
 (u'ice_cream', 49),
 (u'car_sharing', 44),
 (u'waste_disposal', 44),
 (u'motorcycle_parking', 43),
 (u'fire_station', 41),
 (u'veterinary', 41),
 (u'nightclub', 39),
 (u'embassy', 35),
 (u'ci

### Looking for 'religion' in the database

In [17]:
dbUdacity.query_AllResults('''SELECT nodes_tags.value
                                    , count (nodes_tags.key) as Sum    
                                FROM nodes_tags                                    
                                WHERE nodes_tags.key = "religion"                                    
                                GROUP BY nodes_tags.value
                                ORDER BY Sum DESC
                            ;''')

[(u'christian', 51),
 (u'muslim', 23),
 (u'all', 1),
 (u'buddhist', 1),
 (u'hindu', 1),
 (u'scientologist', 1),
 (u'sikh', 1)]

In [18]:
dbUdacity.query_AllResults('''SELECT nodes_tags.value
                                    , count (nodes_tags.key) as Sum    
                                FROM nodes_tags                                    
                                WHERE nodes_tags.key = "religion"                                    
                                GROUP BY nodes_tags.value
                                ORDER BY Sum DESC
                            ;''')

[(u'christian', 51),
 (u'muslim', 23),
 (u'all', 1),
 (u'buddhist', 1),
 (u'hindu', 1),
 (u'scientologist', 1),
 (u'sikh', 1)]

<a id="3"></a>
# Conclusion
***
The OpenStreetMap data of Hamburg, Germany is of fairly reasonable quality. The overall level of systematically completeness very low, e.g. if total numbers of rows for streets are compared to total numbers of rows for postcodes, phone numbers, amenity, restaurant or any other meaningful piece of information.

Furthermore, the type of information requested for a given location is not systematically required during input leading to disadvantages to competitors like GoogleMaps.

In the detailed analysis, the input typos of humans for street names are so rare, that obviously some sort of data cleaning has been already occured. Same for Postal (ZIP) Codes, because there are all of correct digits. The only big issue remains about standardized format for phone numbers.

Although Open Source Community has undertaken a very high level of effort, to provide a very good free map, there are still a lot of gaps compared to competitors like GoogleMaps, e.g.

* indicated public transportation stops & their lines,
* traffic jams,
* touristic attractions,
* company location - including direct hyperlink to homesite etc.

## Additional suggestions & Ideas

* __General level of available Information__

    In general, the available type of information is reasonble non-systematic and seems to depend purely on user
    interest.
    
    For example, if 'nodes_tags'.key = 'amenity', there are 1527 'nodes_tags'.values = 'restaurants' listed. However,
    there is listed as 'nodes_tags'.key = 'restaurant' listed additional further 2 ones. as  keyCompared to other regions in the world, there are absolutely no information given by any of these tags:
    - *'restaurant'* (only 2 rows)
    - *'religion'* (79 rows)
        

* __typos and well-formatted input__

    Typically, post-entering cleaning activities should be avoided as there are easy to implement prevention measures,
    e.g. GUI / CSS format patterns
    
    Usually, directly at input prompting, there needs to be a formatting validation, to cross-check if entered patterns
    are meeting defined format. Therefore, it is necessary only to determine an associated css-classin html5 and a
    stored pattern in the database.
    
    This prevents any user to indicate undesired input. Only validated, well-formatted information will be stored in
    database in the background.
    
    
* __Marketshare__

    Similar to Google, OpenStreetMap should set-up an agreement with manufacturers of mobile devices for pre-loading
    OpenStreetMap and encouraging users to provide systematically more information to pre-defined areas of interest.
    In order not to "overload" users with request, there should be defined first a roll-out concept, which type of
    information first should be requested first.

<a id="4"></a>
# Bibliography & Files
***

- OSM files
    - OpenStreetMap-Hamburg-31.osm
    - OpenStreetMap-Hamburg-Sample.osm


- Python classes & methods
    - class myXML.py
    - class mySQL3dbConn.py
    - module myExtension.py, containing class 'myUnicodeDictWriter' inherited from csv.DictWriter and
      class 'myDict2CSVTransformer'
    - module schema.py
    - __[Sphinx Documentation jfo package](./doc/modules.html)__
    

- csv files
    - nodes.csv
    - nodes_tags.csv
    - ways.csv
    - ways_nodes.csv
    - ways_tags.csv


- SQL files
    - OSM_Create_Tables.sql
    - OSM_Drop_Tables.sql
    - OSM_Import_CSV.sql
    
    
- osmHamburg.db SQL3Lite database


- Readme.md